In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding,default_data_collator
from torch.utils.data import DataLoader

In [2]:
def get_element_datasets(args):
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

    def tokenize_function(examples):
        result["A"] = tokenizer(examples["A"],truncation = True,max_length = 256)
        result["B"] = tokenizer(examples["B"],truncation = True,max_length = 256)
        result["C"] = tokenizer(examples["C"],truncation = True,max_length = 256)
        if tokenizer.is_fast:
            result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
        return result

    dataset = load_dataset("json", data_files="/home/huijie/legal/mymodel/data/SCM.json")
    dataset = dataset.shuffle(seed=42)["train"].rename_column("labels","classification").train_test_split(0.1)
    tokenized_datasets = dataset.map(
        tokenize_function, batched=True, remove_columns=["class","sentence"],num_proc =16
    )
    data_collator = DataCollatorWithPadding(tokenizer,padding= "max_length",max_length = 256,pad_to_multiple_of= 8)
    return tokenized_datasets["train"],tokenized_datasets["test"],data_collator

In [69]:
df = pd.read_json (r'/home/huijie/legal/mymodel/data/SCM.json',lines=True)
dataset = Dataset.from_pandas(df)
stat_a = [len(inp) for inp in df["A"]]
stat_b = [len(inp) for inp in df["B"]]
stat_c = [len(inp) for inp in df["C"]]
stat_label = [len(inp) for inp in df["label"]]
print("A stat")
s = pd.Series(stat_a)
print(s.describe())
print("B stat")
s = pd.Series(stat_b)
print(s.describe())
print("C stat")
s = pd.Series(stat_c)
print(s.describe())
print("label stat")
s = pd.Series(stat_label)
print(s.describe())

A stat
count    27506.000000
mean       677.076965
std         47.875176
min        586.000000
25%        646.000000
50%        675.000000
75%        703.000000
max       1063.000000
dtype: float64
B stat
count    27506.000000
mean       676.187450
std         45.858378
min        586.000000
25%        646.000000
50%        675.000000
75%        701.000000
max       1063.000000
dtype: float64
C stat
count    27506.000000
mean       426.275576
std        328.079648
min          1.000000
25%          1.000000
50%        641.000000
75%        682.000000
max       1063.000000
dtype: float64
label stat
count    27506.000000
mean       251.143641
std        327.567979
min          1.000000
25%          1.000000
50%          1.000000
75%        654.000000
max       1063.000000
dtype: float64


In [73]:
for i,x in enumerate(dataset):
    if len(x["label"]) > 1:
        # min = None
        # min_key = None 
        # switch_key1 = None 
        # for key,item in dataset[i].items():
        #     print("key:{} len: {}".format(key, len(item)) )
        #     min = len(item)
        # if min ==1:
        #     dataset[i][]
        # else:
        #     print("!!!!!!")
        #     print(dataset[i])
        text = dataset[i]["label"]
        label = None
        for _key,item in dataset[i].items():
            if len(item) ==1:
                label = item
        x[label], x["label"]= text,label
        # for key,item in x.items():
        #     print("key:{} len: {}".format(key, len(item)), end="" )

        

In [2]:
# dataset
# print(dataset[0])

# stat_a = [len(inp) for inp in dataset["A"]]
# stat_b = [len(inp) for inp in dataset["B"]]
# stat_c = [len(inp) for inp in dataset["C"]]
# stat_label = [len(inp) for inp in dataset["label"]]
# import pandas as pd
# s = pd.Series(stat_a)
# print(s.describe())
# s = pd.Series(stat_b)
# print(s.describe())
# s = pd.Series(stat_c)
# print(s.describe())
# s = pd.Series(stat_label)
# print(s.describe())
from datasets import Dataset, Value, ClassLabel, Features
features = Features({"text": Value("string"),"label":ClassLabel(num_classes = 29,names=['盜竊罪', '爆竊罪', '猥褻侵犯罪', '侵害人身罪', '不小心駕駛罪', '販運危險藥物罪', '入屋犯法罪', 
            '搶劫罪', '以欺騙手段取得財產罪', '販毒罪', '販運毒品罪', '普通襲擊罪', '洗黑錢罪', '串謀詐騙罪', 
            '處理贓物罪', '刑事恐嚇罪', '串謀勒索罪', '非法入境罪', '強姦罪', '非禮罪', '縱火罪', '扒竊罪', 
            '管有危險藥物罪', '危險駕駛引致他人死亡罪', '襲擊致造成身體傷害罪', '管有虛假文書罪', '使用虛假文書罪',
             '危險駕駛引致他人身體受嚴重傷害罪', '危險駕駛罪']) ,"begin": Value("string")})
dataset = load_dataset("json", data_files="/home/huijie/legal/crime_prediction/data_prepare/crime_prediction.json",features=features)
dataset = dataset["train"]
dataset.features


Using custom data configuration default-511fb7c719a2c1cb
Reusing dataset json (/home/huijie/.cache/huggingface/datasets/json/default-511fb7c719a2c1cb/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=29, names=['盜竊罪', '爆竊罪', '猥褻侵犯罪', '侵害人身罪', '不小心駕駛罪', '販運危險藥物罪', '入屋犯法罪', '搶劫罪', '以欺騙手段取得財產罪', '販毒罪', '販運毒品罪', '普通襲擊罪', '洗黑錢罪', '串謀詐騙罪', '處理贓物罪', '刑事恐嚇罪', '串謀勒索罪', '非法入境罪', '強姦罪', '非禮罪', '縱火罪', '扒竊罪', '管有危險藥物罪', '危險駕駛引致他人死亡罪', '襲擊致造成身體傷害罪', '管有虛假文書罪', '使用虛假文書罪', '危險駕駛引致他人身體受嚴重傷害罪', '危險駕駛罪'], id=None),
 'begin': Value(dtype='string', id=None)}

In [3]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
from transformers import AutoModel
model = AutoModel.from_pretrained("xlm-roberta-base")

model.eval()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLMRobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): 

In [4]:
labels = ['盜竊罪', '爆竊罪', '猥褻侵犯罪', '侵害人身罪', '不小心駕駛罪', '販運危險藥物罪', '入屋犯法罪', 
            '搶劫罪', '以欺騙手段取得財產罪', '販毒罪', '販運毒品罪', '普通襲擊罪', '洗黑錢罪', '串謀詐騙罪', 
            '處理贓物罪', '刑事恐嚇罪', '串謀勒索罪', '非法入境罪', '強姦罪', '非禮罪', '縱火罪', '扒竊罪', 
            '管有危險藥物罪', '危險駕駛引致他人死亡罪', '襲擊致造成身體傷害罪', '管有虛假文書罪', '使用虛假文書罪',
             '危險駕駛引致他人身體受嚴重傷害罪', '危險駕駛罪']
label2id =  {k:v for v,k in enumerate(labels)}
dataset = dataset.align_labels_with_mapping(label2id, "label")


Loading cached processed dataset at /home/huijie/.cache/huggingface/datasets/json/default-511fb7c719a2c1cb/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-6849bb085e66de49.arrow


In [5]:
dataset[:2]

{'text': ['方第一證人需要透過手機作出紀錄。本席裁定控方第一證人當時正在依法執行公務。  陳大律師陳述，控方未能在毫無合理疑點下，證明被告的行為構成阻礙。  高等法院原訟法庭暫委法官黃崇厚（當時官階），在香港特別行政區訴尹明義 案指出，《簡易程序治罪條例》第23條和《侵害人身罪條例》第36（b）條的罪行不同，因為《簡易程序治罪條例》的條文中，沒有故意（wilful）這字眼，所以控方毋須證明被告人故意阻礙執行公務，意即毋須證明阻礙執行公務是被告人進行某行為的意圖（intention）。以《簡易程序治罪條例》第23條這罪行來說，控方',
  ' 沒有任何法例賦予被告權力，容許他當時自行蒐證或執法，因此，縱使他真誠相信他人違反私穩法例，亦不可以强搶他人物品。立法會議員並沒此方面的權力。本席找不到任何法理支持被告搶手機的行為。被告的行為肯定是非法的武力。  基於前述，控方已在毫無合理疑點下，證明所有普通襲擊罪的控罪元素。本席裁定此項控罪罪名成立。 阻礙公職人員執行公務  根據《釋義及通則條例》第3條，公職人員（public officer）指任何在特區政府擔任受薪職位的人。控方第一證人是保安局高級行政主任，必然是公職人員。本席接納案發當日該小組負責在立法會大樓'],
 'label': [3, 11],
 'begin': ['許智峯是立法會議員，可以進入位於中環立法會道1號的立法會綜合大樓，履行公務。 2018年4月24日，立法會大樓會議室1內正舉行《廣深港高鐵條例草案》委員會會議。 當日，政府人員組成一個小組，成員包括保安局高級行政主任梁諾施，及六名運輸及房屋局的人員。他們在立法會大樓不同位置候命，並記錄立法會議員在草案委員會會議舉行時的行蹤。運輸及房屋局助理秘書長運輸3B黎惠珊，是該小組的組長，負責監察及統籌該小組組員；運輸及房屋局助理秘書長運輸3C鄭朗峰，是該小組統籌員；運輸及房屋局二級系統分析／程序編製主任運輸1梁榮燊，',
  '許智峯是立法會議員，可以進入位於中環立法會道1號的立法會綜合大樓，履行公務。 2018年4月24日，立法會大樓會議室1內正舉行《廣深港高鐵條例草案》委員會會議。 當日，政府人員組成一個小組，成員包括保安局高級行政主任梁諾施，及六名運輸及房屋局的人員。他們在立法會大樓不同位置候命，並記錄立法會議員在草案委員會會議舉行時的行蹤。運

In [6]:

def tokenize_function(examples):
    result = tokenizer([examples["begin"],examples["text"]],truncation = True,max_length = 512)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tokenized_datasets = dataset.map(
        tokenize_function, batched=False, remove_columns=["begin","text"],num_proc =16
    )






In [29]:
tokenized_datasets[0]['input_ids'][1]
print(tokenizer.decode(tokenized_datasets[1]["input_ids"][1]))

<s> 沒有任何法例賦予被告權力,容許他當時自行蒐證或執法,因此,縱使他真誠相信他人違反私穩法例,亦不可以强搶他人物品。立法會議員並沒此方面的權力。本席找不到任何法理支持被告搶手機的行為。被告的行為肯定是非法的武力。 基於前述,控方已在毫無合理疑點下,證明所有普通襲擊罪的控罪元素。本席裁定此項控罪罪名成立。 阻礙公職人員執行公務 根據《釋義及通則條例》第3條,公職人員(public officer)指任何在特區政府擔任受薪職位的人。控方第一證人是保安局高級行政主任,必然是公職人員。本席接納案發當日該小組負責在立法會大樓</s>


In [32]:
# label2id = {'B': 0, 'C': 1}
# tokenized_datasets.align_labels_with_mapping(label2id, "a_similar_to")
# tokenized_datasets = tokenized_datasets.rename_column("label","classification")
data_collator = DataCollatorWithPadding(tokenizer,padding= "max_length",max_length = 512,pad_to_multiple_of= 8)
# train_dataloader = DataLoader(tokenized_datasets,8,collate_fn=data_collator)
train_dataloader = DataLoader(tokenized_datasets,8,collate_fn=data_collator)

In [85]:
tokenized_datasets["a_similar_to"][:3]

### define myself con_fucntion 
### 

[1, 1, 0]

In [33]:
for batch in train_dataloader:
    label = batch.pop("label")
    for key,item in batch.items():
        print(key+": " ,item.shape)
    out = model(**batch)
    break

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [49]:
eval_dataloader = DataLoader(vaild,batch_size=4,collate_fn=con_fun) 
eval_dataloader

In [50]:
from tqdm import tqdm

for batch in eval_dataloader:
    a = batch.pop("classification")
    print(a)
    print(batch)
    break
    

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.